### **Union_roads**

In [ ]:
import folium
import pyproj
import momepy
import shapely
import osmnx as ox
import pandas as pd
import networkx as nx
import geopandas as gpd
import matplotlib.pyplot as plt

from tqdm import tqdm
from shapely.wkt import loads, dumps
from shapely.ops import nearest_points
from shapely.geometry import Point, Polygon, LineString, MultiLineString

In [ ]:
def download_osm(id, tags: dict):
    geocode_to_gdf = ox.geocode_to_gdf(id, by_osmid=True)
    polygon_boundary = geocode_to_gdf.unary_union
    gdf = ox.features_from_polygon(polygon_boundary, tags=tags)
    return gdf

In [ ]:
tags = {'highway': ['motorway', 'trunk', 'primary', 
                    'secondary','tertiary', 'unclassified', 
                    'residential', 'motorway_link', 'trunk_link', 
                    'primary_link', 'secondary_link', 'tertiary_link', 
                    'service']}
gdf = download_osm(['R176095', 'R337422'], tags)

In [ ]:
gdf_roads_LO = gpd.read_file('data/geojsons/roads.geojson')

In [ ]:
print(list(gdf_roads_LO.geometry)[0])

In [ ]:
gdf_roads_LO = gdf_roads_LO[gdf_roads_LO['YEAR'].notna()]
gdf_roads_LO = gdf_roads_LO.to_crs(epsg=3857)

In [ ]:
gdf_roads_LO_line = gdf_roads_LO.explode().reset_index(drop=True)

In [ ]:
gdf_part = gpd.read_file('data/geojsons/roads_OSM.geojson')
gdf_part = gdf_part.to_crs(epsg=3857)

In [ ]:
# Создаем LineString объекты
line1 = LineString([(0, 1), (0, 2), (2, 2)])
line2 = LineString([(1, 1), (1, 2), (2, 2), (2, 3)])
line3 = LineString([(3, 1), (3, 2), (2, 2)])
line4 = LineString([(1,4), (4,4)])
# line5 = LineString([(2.8,2), (4,2)])
# line6 = LineString([(3,1), (3,2.2)])
gdf1 = gpd.GeoDataFrame([line1, line2, line3], columns=["geometry"], crs="EPSG:3857")
gdf2 = gpd.GeoDataFrame([line4], columns=["geometry"], crs="EPSG:3857")

In [ ]:
fig, ax = plt.subplots()
gdf1.plot(ax=ax, color='blue')
gdf2.plot(ax=ax, color='red')
plt.show()

In [ ]:
# gdf2 = gpd.GeoDataFrame([line for line in shapely.unary_union(gdf1).geoms], columns=["geometry"], crs="EPSG:3857")

import geopandas as gpd
import matplotlib.pyplot as plt

# Предполагается, что gdf2 уже определен и содержит линии как показано в вашем примере

# Создаем фигуру и оси для отрисовки
fig, ax = plt.subplots()

# Определяем список цветов для линий
colors = ['red', 'green', 'blue', 'orange', 'purple']

# Итерируем по строкам GeoDataFrame и отрисовываем каждую линию отдельно
for idx, row in gdf_result.iterrows():
    # Используем индекс строки для выбора цвета из списка
    # Если индекс превышает длину списка цветов, используем остаток от деления индекса на длину списка
    color = colors[idx % len(colors)]
    
    # Отрисовываем линию с выбранным цветом
    line = row.geometry
    x, y = line.xy
    ax.plot(x, y, color=color)

# Устанавливаем равное соотношение сторон, чтобы геометрические фигуры не искажались
ax.set_aspect('equal')

plt.show()

In [ ]:
print(gdf_result)

In [ ]:
def nearest_point(gdf_part, gdf_roads_LO_line, meters=50):
    # Создаем пустой список для хранения результатов
    geometries = []
    
    # Итерируем по каждой линии в gdf_roads_LO_line['geometry']
    for line in tqdm(gdf_roads_LO_line['geometry']):
        used_points = set()  # Множество для отслеживания использованных точек
        # Создаем временный GeoDataFrame для линии
        temp_gdf = gpd.GeoDataFrame([line], columns=["geometry"], crs="EPSG:3857")
        # Ищем ближайшие точки
        gdf_func = gpd.sjoin_nearest(gdf_part, temp_gdf, max_distance=meters)
        
        # Если нет ближайших точек, просто добавляем линию
        if gdf_func.empty:
            continue
        
        # Проверяем расстояние от обоих концов линии
        start_point = Point(line.coords[0])
        end_point = Point(line.coords[-1])
        start_distances = gdf_part.distance(start_point)
        end_distances = gdf_part.distance(end_point)
        near_start = start_distances[start_distances <= meters].sort_values()
        near_end = end_distances[end_distances <= meters].sort_values()
        
        # Инициализируем новую линию как список координат
        new_line_coords = list(line.coords)
        
        # Получаем индексы ближайших точек, исключая уже использованные
        start_idx = next((idx for idx in near_start.index if idx not in used_points), None)
        if start_idx is not None:
            used_points.add(start_idx)
            point_for_project_start = gdf_part.loc[start_idx].geometry.interpolate(gdf_part.loc[start_idx].geometry.project(start_point))
            new_line_coords.insert(0, (point_for_project_start.x, point_for_project_start.y))
        
        end_idx = next((idx for idx in near_end.index if idx not in used_points), None)
        if end_idx is not None:
            used_points.add(end_idx)
            point_for_project_end = gdf_part.loc[end_idx].geometry.interpolate(gdf_part.loc[end_idx].geometry.project(end_point))
            new_line_coords.append((point_for_project_end.x, point_for_project_end.y))
        
        # Создаем новую линию из обновленного списка координат
        new_line = LineString(new_line_coords)
        geometries.append(new_line)

    # Создаем GeoDataFrame из списка линий
    gdf_result = gpd.GeoDataFrame(geometry=geometries, crs="EPSG:3857")
    # Объединяем с исходным GeoDataFrame
    gdf_result = pd.concat([gdf_result, gdf_part], ignore_index=True)

    return gdf_result

In [ ]:
gdf_result = nearest_point(gdf_part, gdf_roads_LO_line, meters=150)

In [ ]:
gdf_result_2 = gpd.GeoDataFrame([line for line in shapely.unary_union(gdf_result['geometry']).geoms], columns=["geometry"], crs="EPSG:3857")

In [ ]:
gdf_result_2.to_file('gdf_result_2.geojson', driver='GeoJSON')

In [ ]:
graph_lo = momepy.gdf_to_nx(gdf_result.to_crs(epsg=4326))
for node in graph_lo.nodes:
    x = node[0]
    y = node[1]
    graph_lo.nodes()[node]['x'] = x
    graph_lo.nodes()[node]['y'] = y

In [ ]:
import folium

m = folium.Map(tiles='CartoDB Dark_Matter')


# Добавляем рёбра на карту
for edge in graph_lo.edges(data=True):
    # Если у ребра есть атрибут 'geometry', используем его для отображения
    if 'geometry' in edge[2] and edge[2]['geometry'] is not None:
        # Извлекаем координаты из LINESTRING
        linestring = list(edge[2]['geometry'].coords)
        # Преобразуем координаты для folium (переворачиваем их)
        points = [(coord[1], coord[0]) for coord in linestring]
        if isinstance(edge[2]["highway"], str):
            folium.PolyLine(points, color='green', tooltip=f'highway: {edge[2]["highway"]}').add_to(m)
        else:
            folium.PolyLine(points, color='red', tooltip=f'highway: {edge[2]["highway"]}').add_to(m)

# Добавляем узлы на карту
for node, data in graph_lo.nodes(data=True):
    folium.CircleMarker(location=node[::-1], radius=2, color='yellow').add_to(m)


# Отображаем карту
m.save('spb_line.html')

In [ ]:
import folium

# Создаем карту с использованием темной темы CartoDB
m = folium.Map(location=[59.90669247649436, 29.2840537417804], tiles='CartoDB Dark_Matter', zoom_start=15)

# Добавляем рёбра на карту
for edge in graph_2.edges:
    start_node, end_node = edge[0], edge[1]
    points = [start_node[::-1], end_node[::-1]]  # Переворачиваем координаты для folium
    folium.PolyLine(points, color='blue').add_to(m)

# Добавляем узлы на карту
for node in graph_2.nodes:
    folium.CircleMarker(location=node[::-1], radius=2, color='red').add_to(m)

# Отображаем карту
m.save('spb_graph.html')

### **Union_roads_2**
Тут я пробую взять graph и сделать из него gdf

In [2]:
import folium
import pyproj
import momepy
import shapely
import numpy as np
import osmnx as ox
import pandas as pd
import networkx as nx
import geopandas as gpd
import matplotlib.pyplot as plt

from tqdm import tqdm
from shapely.wkt import loads, dumps
from shapely.ops import nearest_points
from shapely.geometry import Point, Polygon, LineString, MultiLineString

d:\IT\GitHub\ITMO_NCCR\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
spb = ox.geocode_to_gdf('R337422', by_osmid=True) # Санкт-Петербург
lo = ox.geocode_to_gdf('R176095', by_osmid=True)  # Ленинградская область

spb_buff = spb.to_crs(epsg=3857).buffer(1000).to_crs(epsg=4326)
lo_buff = lo.to_crs(epsg=3857).buffer(3000).to_crs(epsg=4326)

lo_filter = "['highway'~'motorway|trunk|primary|secondary|tertiary|unclassified|residential|motorway_link|trunk_link|primary_link|secondary_link|tertiary_link']"
graph_lo = ox.graph_from_polygon(lo_buff.union(spb_buff).unary_union, network_type='drive', custom_filter=lo_filter)

d:\IT\GitHub\ITMO_NCCR\.venv\lib\site-packages\osmnx\_overpass.py:245: UserWarning: This area is 52 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


In [4]:
def geometry_type_counts(gdf):
    geom_types_counts = gdf.geometry.geom_type.value_counts().to_dict()
    return geom_types_counts

In [ ]:
mymap = folium.Map(tiles='CartoDB Dark_Matter')

# Создаем FeatureGroup для узлов
edges_group = folium.FeatureGroup(name='Roads')
other_group = folium.FeatureGroup(name='Other_roads')

all_roads_type = ['motorway', 'motorway_link', 'primary', 'primary_link', 'secondary', 'secondary_link', 'tertiary', 'tertiary_link', 'trunk', 'trunk_link']
weights = {
 'motorway':        5, # Автомагистрали 
 'motorway_link':   5, # Съезды на развязках дорог, на которых действуют те же правила движения, что и на (motorway).
 'primary':         3,  # Автомобильные дороги регионального значения
 'primary_link':    3,  # Съезды на развязках дорог с той же важностью в дорожной сети, что и primary.
 'residential':     1,  # Дороги, которые проходят внутри жилых зон, а также используются для подъезда к ним. 
 'secondary':       2,  # Автомобильные дороги областного значения
 'secondary_link':  2,  # Съезды на развязках дорог с той же важностью в дорожной сети, что и secondary.
 'tertiary':        1,  # Более важные автомобильные дороги среди прочих 
                         # автомобильных дорог местного значения, например
                         # соединяющие районные центры с сёлами, а также несколько сёл между собой.
 'tertiary_link':   1,  # Съезды на развязках дорог с той же важностью в дорожной сети, что и tertiary.
 'trunk':           4,  # Важные дороги, не являющиеся автомагистралями
 'trunk_link':      4,  # Съезды на развязках дорог с той же важностью в дорожной сети, что и trunk.
 'unclassified':    1   # Остальные автомобильные дороги местного значения, образующие соединительную сеть дорог.
 }

for u, v, d in graph_lo.edges(data=True):
    if d.get('highway') in all_roads_type:
        folium.PolyLine(
            [[graph_lo.nodes[u]['y'], graph_lo.nodes[u]['x']],
             [graph_lo.nodes[v]['y'], graph_lo.nodes[v]['x']]],
            color="yellow",
            weight=weights[d.get('highway')],  # Толщина линии
            opacity=1,
            tooltip=f"highway: {d.get('highway')}"
        ).add_to(edges_group)
edges_group.add_to(mymap)

for u, v, d in graph_lo.edges(data=True):
    if d.get('highway') not in all_roads_type:
        folium.PolyLine(
            [[graph_lo.nodes[u]['y'], graph_lo.nodes[u]['x']],
             [graph_lo.nodes[v]['y'], graph_lo.nodes[v]['x']]],
            color="yellow",
            weight=0.5,  # Толщина линии
            opacity=1,
            tooltip=f"highway: {d.get('highway')}"
        ).add_to(other_group)
other_group.add_to(mymap)

# Добавляем контроллер слоев
folium.LayerControl().add_to(mymap)

# Сохраняем карту в HTML файл
mymap.save('data/html/graph_osm_lo_spb.html')

In [ ]:
# ox.save_graphml(graph_lo, 'data/graphml/graph_osm_lo_spb.graphml')

In [2]:
graph_lo = nx.read_graphml('data/graphml/graph_osm_lo_spb.graphml')

In [12]:
# list(graph_lo.nodes(data=True))[0], list(graph_lo.edges(data=True))[0]
try: 
    n, e = ox.graph_to_gdfs(graph_lo)
    print(geometry_type_counts(n), geometry_type_counts(e)) 
except TypeError as e: 
    print("An error occurred while converting graph to GeoDataFrames.") 
    print("Error message: ", str(e))

{'Point': 124602} {'LineString': 308881}


In [ ]:
e[['highway', 'geometry']][e['highway'] == 'unclassified'].explore()

In [57]:
poly_gdf = gpd.GeoDataFrame(geometry=spb.to_crs(epsg=3857).buffer(-3100))
poly_gdf['polygon_id'] = 1
poly_gdf.set_index('polygon_id', inplace=True)

In [77]:
# Соединение
e_joined = gpd.sjoin(e.to_crs(epsg=3857), poly_gdf, predicate='within')

In [91]:
e_joined['highway']

array(['secondary', 'tertiary', 'residential', 'unclassified', 'motorway',
       'tertiary, unclassified', 'primary', 'secondary_link',
       'residential, unclassified', 'primary_link', 'trunk_link', 'trunk',
       'tertiary_link', 'motorway_link', 'primary_link, primary',
       'secondary_link, secondary', 'secondary_link, unclassified',
       'tertiary, residential', 'motorway_link, primary_link',
       'secondary, primary', 'secondary_link, tertiary_link',
       'motorway_link, motorway', 'unclassified, secondary_link',
       'residential, tertiary_link'], dtype=object)

In [ ]:
test = ['secondary', 'motorway',
       'primary', 'secondary_link',
       'primary_link', 'trunk_link', 'trunk',
       'motorway_link']
e_joined[['highway', 'geometry']][e['highway'].isin(test)].explore()

In [100]:
# Удаление строк
e_new = e.drop(e_joined[~e_joined['highway'].isin(test)].index)
# Создание нового графа из обновленных GeoDataFrames
new_graph = ox.graph_from_gdfs(n, e_new.to_crs(epsg=3857))

In [103]:
mymap = folium.Map(tiles='CartoDB Dark_Matter')

# Создаем FeatureGroup для узлов
edges_group = folium.FeatureGroup(name='Roads')
tertiary_group = folium.FeatureGroup(name='Tertiary')
unclassified_group = folium.FeatureGroup(name='Unclassified')

all_roads_type = ['motorway', 'motorway_link', 'primary', 'primary_link', 'secondary', 'secondary_link', 'trunk', 'trunk_link']
tertiary_type = ['tertiary', 'tertiary_link']
weights = {
 'motorway':        5, # Автомагистрали 
 'motorway_link':   5, # Съезды на развязках дорог, на которых действуют те же правила движения, что и на (motorway).
 'primary':         3,  # Автомобильные дороги регионального значения
 'primary_link':    3,  # Съезды на развязках дорог с той же важностью в дорожной сети, что и primary.
 'residential':     1,  # Дороги, которые проходят внутри жилых зон, а также используются для подъезда к ним. 
 'secondary':       2,  # Автомобильные дороги областного значения
 'secondary_link':  2,  # Съезды на развязках дорог с той же важностью в дорожной сети, что и secondary.
 'tertiary':        1,  # Более важные автомобильные дороги среди прочих 
                         # автомобильных дорог местного значения, например
                         # соединяющие районные центры с сёлами, а также несколько сёл между собой.
 'tertiary_link':   1,  # Съезды на развязках дорог с той же важностью в дорожной сети, что и tertiary.
 'trunk':           4,  # Важные дороги, не являющиеся автомагистралями
 'trunk_link':      4,  # Съезды на развязках дорог с той же важностью в дорожной сети, что и trunk.
 'unclassified':    1   # Остальные автомобильные дороги местного значения, образующие соединительную сеть дорог.
 }

for u, v, d in new_graph.edges(data=True):
    if d.get('highway') in all_roads_type:
        folium.PolyLine(
            [[new_graph.nodes[u]['y'], new_graph.nodes[u]['x']],
             [new_graph.nodes[v]['y'], new_graph.nodes[v]['x']]],
            color="yellow",
            weight=weights[d.get('highway')],  # Толщина линии
            opacity=1,
            tooltip=f"highway: {d.get('highway')}"
        ).add_to(edges_group)
    elif d.get('highway') in tertiary_type:
        folium.PolyLine(
            [[new_graph.nodes[u]['y'], new_graph.nodes[u]['x']],
             [new_graph.nodes[v]['y'], new_graph.nodes[v]['x']]],
            color="yellow",
            weight=weights[d.get('highway')],  # Толщина линии
            opacity=1,
            tooltip=f"highway: {d.get('highway')}"
        ).add_to(tertiary_group)
    else:
        folium.PolyLine(
            [[new_graph.nodes[u]['y'], new_graph.nodes[u]['x']],
             [new_graph.nodes[v]['y'], new_graph.nodes[v]['x']]],
            color="yellow",
            weight=0.5,  # Толщина линии
            opacity=1,
            tooltip=f"highway: {d.get('highway')}"
        ).add_to(unclassified_group)

edges_group.add_to(mymap)
tertiary_group.add_to(mymap)
unclassified_group.add_to(mymap)

folium.GeoJson(
    lo.unary_union.boundary,
    name='OSM',
    style_function=lambda feature: {
        'color': 'green',        # Цвет границы
        'weight': 1.5,           # Толщина границы
    }
).add_to(mymap)

# Добавляем контроллер слоев
folium.LayerControl().add_to(mymap)

# Сохраняем карту в HTML файл
mymap.save('data/html/graph-osm_lo_spb_without-unclassified.html')